In [1]:
import chromadb
from chromadb.utils import embedding_functions
import json
from typing import Dict, List
import uuid

class ExemplarRetriever:
    def __init__(self):
        # Initialize ChromaDB client
        self.client = chromadb.Client()
        
        # Use default embedding function (all-MiniLM-L6-v2)
        self.embedding_fn = embedding_functions.DefaultEmbeddingFunction()
        
        # Get or create collection for exemplars
        self.collection = self.client.get_or_create_collection(
            name="exemplars",
            embedding_function=self.embedding_fn
        )

    def reset_collection(self):
        """Delete existing collection and create a new one"""
        try:
            self.client.delete_collection("exemplars")
        except:
            pass
        self.collection = self.client.create_collection(
            name="exemplars",
            embedding_function=self.embedding_fn
        )

    def add_exemplars(self, exemplars: List[Dict]):
        """Add multiple exemplars to the collection"""
        # First, reset the collection to start fresh
        self.reset_collection()
        
        ids = [str(uuid.uuid4()) for _ in exemplars]
        queries = [ex["query"] for ex in exemplars]
        metadatas = [
            {
                "rationale": ex["rationale"],
                "json_output": json.dumps(ex["json_output"])
            } 
            for ex in exemplars
        ]
        
        # Add to collection
        self.collection.add(
            ids=ids,
            documents=queries,
            metadatas=metadatas
        )

    def query_exemplars(self, 
                       query: str, 
                       n_results: int = 3) -> List[Dict]:
        """Query similar exemplars"""
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        
        # Process results
        exemplars = []
        for idx in range(len(results['ids'][0])):
            exemplar = {
                "query": results['documents'][0][idx],
                "rationale": results['metadatas'][0][idx]['rationale'],
                "json_output": json.loads(results['metadatas'][0][idx]['json_output'])
            }
            exemplars.append(exemplar)
            
        return exemplars




In [2]:
import json

with open('generated_input_output_pairs_v2.json') as f:
    gt_data_list = json.load(f)

In [ ]:
len(gt_data_list)

In [4]:
import random

def randomize_queries(queries_list, seed=42, percentage=0.8):
    # create a list
    new_list = queries_list
    # extend the list with all the lists except the 'sample_data'
    # for query in queries_list:
    #     if query != 'sample_data':
    #         new_list.extend(queries_list[query])
    # randomize the list using a seed
    random.seed(42)
    random.shuffle(new_list)

    # add code for splitting it in three test, train, and validation with a percentage of perecentage
    train = new_list[:int(len(new_list)*percentage)]
    test = new_list[int(len(new_list)*percentage):int(len(new_list)*percentage)+int(len(new_list)*(1-percentage)/2)]
    validation = new_list[int(len(new_list)*percentage)+int(len(new_list)*(1-percentage)/2):]


    return train, test, validation

In [5]:
train, test, validation = randomize_queries(gt_data_list)

In [ ]:
len(train)

In [ ]:
import time

# Initialize retriever
retriever = ExemplarRetriever()
t1 = time.time()
# Add exemplars to store
retriever.add_exemplars(train)
print(f"Index time: {time.time() - t1:.2f} seconds")


In [ ]:
query = test[15]['query']
print(query)

In [ ]:
retriever.query_exemplars(query=query)

In [ ]:
from together import Together
from dotenv import load_dotenv
import os

_ = load_dotenv('../.env')

print(os.environ.get('TOGETHER_API_KEY'))

In [ ]:
messages = []

In [71]:
SYSTEM_L = """You are an AI assistant specialized in extracting management levels and job titles from queries. Your task is to analyze a given query and identify relevant management levels and job titles."""

USER_L = """
First, review these similar examples for inspiration:

<similar_examples>
{{SIMILAR_EXAMPLES}}
</similar_examples>

Now, consider the following query:

<query>
{{QUERY}}
</query>

Using the examples as a guide, analyze the query and extract the relevant management levels and job titles. Consider the following management levels:

['Partners,' 'Founder or Co-founder,' 'Board of Directors,' 'CSuite/Chiefs,' 'Executive VP or Sr. VP,' 'General Manager,' 'Manager,' 'Head,' 'Senior Partner,' 'Junior Partner,' 'VP,' 'Director,' 'Senior (All Senior-Level Individual Contributors),' 'Mid (All Mid-Level Individual Contributors),' 'Junior (All Junior-Level Individual Contributors)']

Guidelines for analysis:
1. Identify potential management levels and job titles in the query.
2. Normalize any abbreviations (e.g., CEO → Chief Executive Officer, VP → Vice President).
3. Separate management levels from job titles.
4. Exclude company names, locations, or other irrelevant information.

Before providing your final output, wrap your thought process in <analysis> tags. Focus on explaining your extraction and categorization choices rather than comparing directly to the examples. In your analysis:

1. List all potential management levels and job titles found in the query.
2. Explain your reasoning for categorizing each item as either a management level or job title.
3. Keep it moderate 


Your final output should follow this format:

<analysis>
[Your detailed thought process for extracting and categorizing the information]
</analysis>

<json_output>
{
    "management_levels": [],
    "titles": []
}
</json_output>

Remember to be thorough in your analysis but concise in your final output."""

In [86]:
def get_prompt(query, retriever):
        
    from jinja2 import Template
    exemplars = retriever.query_exemplars(query=query)
    user = Template(USER_L).render({
        "SIMILAR_EXAMPLES" : exemplars,
        "QUERY" : query
    })
    return [
        {'role' : 'system', 'content' : SYSTEM_L},
        {'role': 'user' , 'content' : user}
    ], exemplars

In [61]:
import re
import json

def extract_json_from_tags(text):
    # Use re to find the JSON string between <json_output> tags
    match = re.search(r'<json_output>(.*?)</json_output>', text, re.DOTALL)
    if match:
        json_str = match.group(1)
        # Convert the JSON string to a Python dictionary
        json_dict = json.loads(json_str)
        return json_dict
    else:
        return None

In [65]:
exemplar = test[25]

In [38]:
messages = get_prompt(exemplar['query'], retriever)

In [39]:
client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K",
    messages=messages,
    max_tokens=1024,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<|eot_id|>","<|eom_id|>"],
    stream=False
)
for token in response:
    if hasattr(token, 'choices'):
        print(token.choices[0].delta.content, end='', flush=True)

In [41]:
print(exemplar['json_output'])
print(response.choices[0].message.content)

{'management_levels': [], 'titles': ['Growth Hacker']}
<analysis>
1. **Potential Management Levels and Job Titles**:
   - Growth hackers
   - Edtech startups
   - K-12
   - India

2. **Normalization**:
   - No abbreviations need to be normalized in this query.

3. **Categorization**:
   - **Management Levels**: None of the terms directly map to the predefined management levels. However, "growth hackers" implies a leadership role, but the term "hackers" is not typically associated with a specific management level.
   - **Job Titles**: "Growth hackers" is a job title that refers to a specific role, which is a leadership position focused on growth within an organization. The term "growth hackers" is a common job title in the industry.

4. **Extraction and Categorization Choices**:
   - I chose to categorize "growth hackers" as a job title because it specifies a role and is a common job title in the industry. The term "growth" implies a focus on leadership and growth within an organization

In [68]:
def get_together_api_response(exemplar, retriever):

    t1 = time.time()
    messages = get_prompt(exemplar['query'], retriever)
    t2 = time.time()
    client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))

    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K",
        messages=messages,
        max_tokens=1024,
        temperature=0.1,
        stop=["<|eot_id|>","<|eom_id|>"],
        stream=False
    )
    t3 = time.time()
    response_json = extract_json_from_tags(response.choices[0].message.content)
    return {
        "query" : exemplar['query'],
        "ground_truth" : exemplar['json_output'],
        "prediction" : response_json,
        "exact_match" : exemplar['json_output'] == response_json,
        "exemplar_retrieval_latency" : float(str(t2-t1)[:5]),
        "request_latency" : float(str(t3-t2)[:5])
    }

    

In [72]:
result = get_together_api_response(exemplar, retriever)

In [73]:
result

{'query': 'Associate Director Biostatistics or Senior RWE Data Scientist in pharmaceutical background in United States',
 'ground_truth': {'management_levels': [],
  'titles': ['Associate Director Biostatistics', 'Senior RWE Data Scientist']},
 'prediction': {'management_levels': ['Associate Director'],
  'titles': ['Biostatistics', 'Senior RWE Data Scientist']},
 'exact_match': False,
 'exemplar_retrieval_latency': 0.224,
 'request_latency': 6.289}

In [87]:
import asyncio
import time
import os
import aiohttp
# from your_module import Together  # Assuming Together is synchronous

async def fetch_completion(api_key, messages, model, max_tokens, temperature, stop):
    url = 'https://api.together.xyz/chat/completions' # Replace with the actual endpoint
    headers = {'Authorization': f'Bearer {api_key}', 'Content-Type': 'application/json'}
    payload = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload, headers=headers) as response:
            response_data = await response.json()
            return response_data

async def get_together_api_response_async(exemplar, retriever):
    t1 = time.time()
    messages, exemplars_chunks = get_prompt(exemplar['query'], retriever)
    t2 = time.time()

    # Perform the fetch asynchronously
    response_data = await fetch_completion(
        os.environ.get('TOGETHER_API_KEY'),
        messages,
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K",
        max_tokens=1024,
        temperature=0.1,
        stop=["<|eot_id|>", "<|eom_id|>"]
    )
    t3 = time.time()

    # Extract your JSON accordingly
    response_json = extract_json_from_tags(response_data['choices'][0]['message']['content'])

    return {
        "query": exemplar['query'],
        "ground_truth": exemplar['json_output'],
        "prediction": response_json,
        "exact_match": exemplar['json_output'] == response_json,
        "exemplar_retrieval_latency": float(str(t2-t1)[:5]),
        "request_latency": float(str(t3-t2)[:5]),
        "exemplars" : exemplars_chunks
    }

async def handle_batch(exemplars, retriever):
    tasks = [get_together_api_response_async(exemplar, retriever) for exemplar in exemplars]
    return await asyncio.gather(*tasks)

async def batched_call(exemplars, retriever):
    results = []
    batch_size = 10
    for i in range(0, len(exemplars), batch_size):
        print(f"Processing Batch {i}... ")
        batch = exemplars[i:i + batch_size]
        results.extend(await handle_batch(batch, retriever))
    return results

# Use await in the Jupyter Notebook cell
# results = await batched_call(test, retriever)

In [75]:
test.extend(validation)

In [78]:
len(test), len(validation)

(115, 58)

In [80]:

results = await batched_call(test, retriever)

In [81]:
len(results)

115

In [85]:
count = 0
for item in results:
    print(item)
        

{'query': 'Find HR managers at AI/ML companies with a focus on scaling teams in Canada.', 'ground_truth': {'management_levels': [], 'titles': ['HR Manager', 'Human Resources Manager']}, 'prediction': {'management_levels': [], 'titles': ['HR Manager', 'HR Director', 'HR Vice President']}, 'exact_match': False, 'exemplar_retrieval_latency': 1.149, 'request_latency': 3.472}
{'query': '"I’m interested in people holding senior engineering roles right now, especially those at Amazon with experience in cloud technologies', 'ground_truth': {'management_levels': [], 'titles': ['Senior Engineer', 'Senior Engineering Manager']}, 'prediction': {'management_levels': [], 'titles': ['Senior Engineering Roles']}, 'exact_match': False, 'exemplar_retrieval_latency': 0.098, 'request_latency': 6.747}
{'query': '"Searching for a Regional General Manager in the Lesser Antilles to oversee business operations and growth strategies."', 'ground_truth': {'management_levels': [], 'titles': ['Regional General Mana

0

# Loading Fine Tuned Model results

In [ ]:
import time
from jinja2 import Template
def get_prompt_company_generation(query):
    SYSTEM = """Your name is Lisa and you are intelligent assistant tasked with listing down companies according to mentioned requirements. Make sure to strictly follow the output format."""
    USER = """<Instructions>
            - Based on the given prompt generate companies/institutions/organizations based on the context of the query.
            **Case: 1**
                - If a tasks mentions specific company names or nouns only, generate those. You are to consider nouns as company names even if you're unfamiliar with them. If they refer to some product or something you are sure isnt a company, they belong to case 4.
                - Any noun mentioned in the prompt is to be considered a company word for word. If you don't know that particular company just give "company name~location" in your output.
                    e.g. For the prompt "Apophis"
                        **Good Output**: "Apophis~United States "
                        **Bad Output**: "Apophis~United States", "Apple~United States" ...
                        **Bad Output**: "ApophisTech~United States", "ApophisFinance~United States" ...
                - This case only caters when only company name(s) is present and not the industry or relevant terms. In case of industry, refer below cases
            **Case: 2**
                - If the task requires or mentions lists of companies or similar companies (this also caters "companies like") always try to achieve 50 companies.
            **Case: 3**
                - If the task has company names along with specificly mentioned requirement for a list.
                **Case: 3.1**
                    - If the prompt asks for companies and thier similar companies, Generate those names and then the rest of the list.
                **Case: 3.2**
                    - If the prompt explicitly only asks for companies similar to the names, Only generate the list without those names.
            **Case: 4**
                - If the prompt has any mention that isnt from above but companies can get generated for them, generate.
        </Instructions>
        <Output Format>
            - The section after the thought process where the companies are listed down must be enclosed in xml tag of "Companies". The system will fail otherwise
            - Below is the example of required tags in list section:
            <Companies>
                1. Company Name~Location
                ...
            </Companies>
        </Output Format>
        <Important>
            - Generate the most commonly used or known names for the companies. Don't add things like LLC, Ltd, Inc etc.
            - Location can only be country names. If you don't know the location just add 'United States'.
            - Always treat individual company requirements separately. E.g. "Companies with $500M-$2B in revenue and healthcare companies." here you need to generate companies with $500M–$2B in revenue and companies from healthcare seperately.
            - Once you've given me a company don't give that company again!
            - Case should be decided without considering the sizes requirements mentioned by the user.
            - Dont mention the output format or tags anywhere in the thought process.
            - Even if its being asked not to include a company, return that name as well.
        </Important>
        <Perform Task>
            - Take a deep breath and understand the instructions. Then tell your thought process and only then generate.
            - Generate a numbered list containing company name and it's location separated by a delimiter '~' Keeping the mentioned prompt in view.
            - After your thought process, add a tag of <Companies> and then generate the list of companies. failing to do so would fail the system.
        </Perform Task>
        <Prompt>
        {{query}}
        </Prompt>
        """
    user = Template(USER).render({"query": query})
    messages = [{'role' : 'system', 'content' : SYSTEM},{'role':'user', 'content': user}]
    
    
import time
import os
from together import Together

async def get_together_api_response_async_v2(query):
    t1 = time.time()
    messages = get_prompt_company_generation(query)
    # messages = get_prompt_brief(exemplar['query'], )
    # print(messages)
    t2 = time.time()

    client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))

    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K",
        messages=messages,
        max_tokens=1024,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
        stream=False
    )
    
    t3 = time.time()
import asyncio
import aiohttp
import time
from typing import List, Dict
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Your queries list
queries = [
    "Vice President of IT and Technology at logistics and distribution companies based in the United States with a revenue of over $25B and pretend you are a financial analyst and technical. All firms that are target companies for Central Garden & Pet in their search for a CFO of their pet segment business.",
    "Find me marketing leaders in fintech startups focused on customer acquisition in Southeast Asia.",
    "I need product managers in e-commerce platforms with experience in scaling marketplaces in the U.S.",
    "Identify UX/UI designers who specialize in mobile app design for fintech applications in Europe.",
    "I'm looking for CTOs at blockchain companies with expertise in Web3 and DeFi in Singapore.",
    "Find senior data scientists in healthcare AI startups based in Canada.",
    "I need sales directors at SaaS companies focused on B2B enterprise sales in the U.K.",
    "Find CPOs (Chief Product Officers) in cybersecurity startups with experience in building B2C products in the U.S.",
    "Identify head of customer success roles in SaaS-based companies with a focus on AI/ML products in California.",
    "Find growth marketing leaders at edtech platforms targeting emerging markets like India or Brazil.",
    "I'm looking for CFOs with fundraising experience in climate tech startups based in Europe.",
    "Identify VPs of Engineering in telemedicine startups in the Asia-Pacific region.",
    "I need blockchain developers who have worked on decentralized finance (DeFi) projects in Africa.",
    "Find machine learning engineers working on computer vision in autonomous vehicle companies in the U.S.",
    "Identify HR leaders in SaaS companies who have experience scaling remote teams in Europe.",
    "I need C-suite executives with expertise in scaling SaaS platforms in Latin America.",
    "Find operations managers at logistics startups using AI for route optimization in Southeast Asia.",
    "I'm looking for business development leaders in AI/ML SaaS companies that focus on healthcare in Canada.",
    "Identify CTOs with experience in developing AR/VR platforms for retail companies in Europe.",
    "Find CEOs with experience in growing SaaS startups from Series A to IPO in the U.S.",
    "I need heads of sales in financial services companies focusing on AI/ML solutions in New York.",
    "Find people for the COO role at deep tech AI companies based in Japan.",
    "Identify marketing heads at wellness tech startups with experience in D2C channels in California.",
    "I'm looking for chief legal officers in cybersecurity firms in Israel.",
    "Find engineering leaders at mobility-as-a-service (MaaS) companies in Europe.",
    "Identify experts in digital transformation for energy sector SaaS platforms in the Nordics.",
    "I need people for the VP of Operations role in B2B SaaS companies based in Singapore.",
    "Find product marketing managers in medtech startups in the U.S.",
    "Identify heads of corporate strategy at AI-driven SaaS companies focused on fintech in Europe.",
    "I'm looking for CROs (Chief Revenue Officers) at climate tech startups based in Australia."
]

async def shorten_prompt(session: aiohttp.ClientSession, query: str) -> Dict:
    """Async function to call the shortening API"""
    url = "https://qlu2-ai-backend-7wzmpo3asa-uc.a.run.app/shorten_prompt"
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    payload = {
        "prompt": query,
        "agent": "dual"
    }
    
    try:
        async with session.post(url, headers=headers, json=payload) as response:
            return await response.json()
    except Exception as e:
        logger.error(f"Error in shorten_prompt: {str(e)}")
        return None

async def together_api_response(query: str) -> Dict:
    """Async function to call the Together API"""
    # Replace with your actual Together API implementation
    t1 = time.time()
    
    t2 = time.time()
    return {"status": "success", "query": query, "response": response}

async def process_query(session: aiohttp.ClientSession, query: str, semaphore: asyncio.Semaphore):
    """Process a single query with timing"""
    async with semaphore:
        start_time = time.time()
        try:
            # Get shortened prompt
            shortened = await shorten_prompt(session, query)
            if not shortened:
                return {"query": query, "status": "failed", "time": time.time() - start_time}

            # Process both current and past
            tasks = []
            for key in ['current', 'past']:
                if shortened.get(key):
                    tasks.append(together_api_response(session, shortened[key]))
            
            if tasks:
                results = await asyncio.gather(*tasks)
            
            return {
                "query": query,
                "status": "success",
                "time": time.time() - start_time,
                "results": results if tasks else None
            }
        except Exception as e:
            logger.error(f"Error processing query '{query}': {str(e)}")
            return {"query": query, "status": "failed", "error": str(e)}

async def process_all_queries(queries: List[str], concurrent_limit: int = 5):
    """Process all queries concurrently"""
    async with aiohttp.ClientSession() as session:
        semaphore = asyncio.Semaphore(concurrent_limit)
        tasks = [process_query(session, query, semaphore) for query in queries]
        return await asyncio.gather(*tasks)

# Function to run in Jupyter
async def run_processing():
    start_time = time.time()
    results = await process_all_queries(queries)
    total_time = time.time() - start_time
    
    # Print summary
    successful = sum(1 for r in results if r['status'] == 'success')
    print(f"\nProcessing Summary:")
    print(f"Total queries: {len(results)}")
    print(f"Successful: {successful}")
    print(f"Failed: {len(results) - successful}")
    print(f"Total time: {total_time:.2f} seconds")
    
    return results

# For running in Jupyter notebook
def process_queries():
    return asyncio.run(run_processing())